<a href="https://colab.research.google.com/github/meashu31/AIML-Capstone-CV9/blob/Vadhi_master/Test_Model_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Gogle Drive

In [ ]:
pip install segmentation-models

In [ ]:
pip install plot_keras_history

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [4]:
#------------------------- Load necessary libraries -------------------------#
####------------------------- for loading path -------------------------#
import os
import pandas as pd
import pickle
from tqdm import tqdm
from matplotlib import image
from matplotlib import pyplot
from plot_keras_history import plot_history
####------------------------- for DataGenerator -------------------------#
import numpy as np
import keras
import tensorflow as tf
from random import choice
from keras.preprocessing.image import ImageDataGenerator
####------------------------- for Plotting -------------------------#
import matplotlib.pyplot as plt
####------------------------- for Model -------------------------#
from keras.utils.layer_utils import count_params
from tabulate import tabulate
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import *
from keras.layers import  *
import tensorflow as tf

import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter

IMAGE_HEIGHT=224
IMAGE_WIDTH=224
batch_size = 8
epoch = 10

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [5]:
#------------------------- Declare required path -------------------------#
ProjectPath = '/content/drive/My Drive/great learning/capstone project/rsna-pneumonia-detection/'
CSVPath = os.path.join(ProjectPath,"DataSet_CSV")
RawImg = os.path.join(ProjectPath,'data/image_data/images')
RawMask = os.path.join(ProjectPath,'data/mask_data/masks')
balancedImg = os.path.join(ProjectPath,"Balanced_Data")
ImageFolder = os.path.join(ProjectPath,"Balanced_Data/Images")
MaskFolder = os.path.join(ProjectPath,"Balanced_Data/Mask")
NormalImg = os.path.join(ImageFolder,'Normal')
NormalMask = os.path.join(MaskFolder,'Normal')
PneumoniaImg = os.path.join(ImageFolder,'Pneumonia')
PneumoniaMask = os.path.join(MaskFolder,'Pneumonia')
os.chdir(ProjectPath) #changing the current directory path to the project direcotry where the dataset is available
#------------------------- Load CSV to get label info -------------------------#

In [6]:
print(len(os.listdir(NormalImg)))
print(len(os.listdir(PneumoniaMask)))

20672
6012


In [7]:
bbox_dataframe = pd.read_csv(os.path.join(CSVPath,"stage_2_train_labels.csv"), usecols=['patientId','Target'])
bbox_dataframe.sort_values("patientId", inplace=True)
bbox_dataframe.drop_duplicates( keep="first", inplace=True)

def append_ext(fn):
    return fn+".png"

def TargetUpdate(fn):
  return 'Normal' if fn == 0 else "Normal,Pneumonia"

bbox_dataframe['patientId'] = bbox_dataframe['patientId'].apply(append_ext)
bbox_dataframe['Target'] = bbox_dataframe['Target'].apply(TargetUpdate)

bbox_dataframe.head(5)

,patientId,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6.png,Normal
28989,000924cf-0f8d-42bd-9158-1af53881a557.png,Normal
28990,000db696-cf54-4385-b10b-6b16fbb3f985.png,"Normal,Pneumonia"
28992,000fe35a-2649-43d4-b027-e67796d412e0.png,"Normal,Pneumonia"
28994,001031d9-f904-4a23-b3e5-2c088acd19c6.png,"Normal,Pneumonia"


## Backboned-UNet

In [8]:
def my_image_mask_generator(image_data_generator, mask_data_generator):
    train_generator = zip(image_data_generator, mask_data_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

In [9]:
def dice_coefficient(y_true, y_pred):
    #### Add your code here ####
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())#### Add your code here ####/

In [15]:
def create_model(Backbone='vgg16',pretrained=True,ShowSummery=False,TrainModel=True,ImageGeneratorFlow='From_Folder',TestModel=True):
  # Data Generator
  preprocess_input = get_preprocessing(Backbone)
  seed = 66
  # define model
  print("-------------------Generating UNet Model----------------------")
  model = Unet(Backbone, encoder_weights='imagenet', input_shape=(224, 224, 3))
  ##-------------------getLast layer and add reshape----------------------
  OneThrird = int(len(model.layers)/3)
  for layer in model.layers[:OneThrird]:
      layer.trainable=True
  for layer in model.layers[OneThrird:]:
      layer.trainable=True

  # Model Overview
  trainable_count = count_params(model.trainable_weights)
  non_trainable_count = count_params(model.non_trainable_weights)
  print()
  print(tabulate([['Model', 'UNet'],\
                          ['BackBone', Backbone],\
                          ['Number of Layers', len(model.layers)],\
                          ['Total params', model.count_params()],\
                          ['Trainable params', trainable_count],\
                          ['Non-trainable params', non_trainable_count]],\
                          headers=['Header', 'Value'], tablefmt='orgtbl'))
  print()
  #If Choosen Show Summery
  
  if ShowSummery: 
    print("-------------------Model Summery----------------------")
    print()
    model.summary()
  if TrainModel:
    print("-------------------Training Model----------------------")
    print()
        # this is the augmentation configuration we will use for training
    train_img_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
    train_mask_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
    # this is the augmentation configuration we will use for testing: only rescaling
    val_img_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)
    val_mask_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

    general_args = dict(class_mode=None,target_size=(224,224),batch_size=batch_size,shuffle=True,seed=seed)
    # this is a generator that will read pictures found in subfolers of 'data/train', and indefinitely generate batches of augmented image data
    if ImageGeneratorFlow == 'From_Folder':
      #------------------------- Train DataSetGenerator -------------------------#
      train_img_generator = train_img_datagen.flow_from_directory('Balanced_Data/Images',color_mode="rgb",subset='training',**general_args)
      train_mask_generator = train_mask_datagen.flow_from_directory('Balanced_Data/Mask',color_mode="grayscale",subset='training',**general_args)
      #------------------------- Validation DataSetGenerator -------------------------#
      validation_img_generator =  val_img_datagen.flow_from_directory('Balanced_Data/Images',color_mode="rgb",subset='validation',**general_args)
      validation_mask_generator =  val_mask_datagen.flow_from_directory('Balanced_Data/Mask',color_mode="grayscale",subset='validation',**general_args)
    if ImageGeneratorFlow == 'From_DataFrame':
      # this is a generator that will read pictures found in subfolers of 'data/train', and indefinitely generate batches of augmented image data
      #------------------------- Train DataSetGenerator -------------------------#
      train_img_generator = train_img_datagen.flow_from_dataframe(dataframe=bbox_dataframe,directory=RawImg,x_col="patientId",color_mode="rgb",subset='training',**general_args)
      train_mask_generator = train_mask_datagen.flow_from_dataframe(dataframe=bbox_dataframe,directory=RawMask,x_col="patientId",color_mode="grayscale",subset='training',**general_args)
      #------------------------- Validation DataSetGenerator -------------------------#
      validation_img_generator = val_img_datagen.flow_from_dataframe(dataframe=bbox_dataframe,directory=RawImg,x_col="patientId",color_mode="rgb",subset='validation',**general_args)
      validation_mask_generator = val_mask_datagen.flow_from_dataframe(dataframe=bbox_dataframe,directory=RawMask,x_col="patientId",color_mode="grayscale",subset='validation',**general_args)

    train_generator = my_image_mask_generator(train_img_generator, train_mask_generator)
    validation_generator = my_image_mask_generator(validation_img_generator, validation_mask_generator)

    # Compile model
    Adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-5, amsgrad=False)
    model.compile(loss="binary_crossentropy", optimizer='Adam', metrics=[['accuracy',
                                                                          tf.keras.metrics.MeanIoU(num_classes=2),dice_coefficient]])
    # checkpoint
    filepath = '/content/drive/My Drive/great learning/capstone project/rsna-pneumonia-detection/UNet_Model_Weights/' + Backbone + "/"
    checkpoint = ModelCheckpoint(filepath+"model-{loss:.2f}.h5", monitor="loss", verbose=1, save_best_only=True, save_weights_only=True, mode="min", period=1)
    stop = EarlyStopping(monitor="loss", patience=5, mode="min")
    print()
    history = model.fit(train_generator,
                    epochs=epoch,
                    use_multiprocessing = True, workers = 15,
                    steps_per_epoch = 21348//batch_size,
                    validation_data=validation_generator,
                    validation_steps = 5336//batch_size,
                    callbacks= [checkpoint, stop]).history
    #--------------------- Plot Loss and Accuracy --------------
    print()
    print('-------------------Model training over Epochs----------------------')
    print()
    plot_history(history)
    plt.show()
    now = datetime.now() # current date and time
    date_time = now.strftime("%m-%d-%Y-%H-%M-%S")
    plot_history(history, path=filepath + "Performance_" + date_time + ".png")
    plt.close()
  #---------------- DO NOT USE TEST AS OF NOW --------------------
  if TestModel:
    print()
    print('-------------------Model performance----------------------')
    print()
    #loss, acc, IOU = model.evaluate_generator(generator=validation_generator, steps=50)
    print()
    #print('Model Loss :',loss)
    #print('Accuracy Loss :',acc)
    #print('IOU Score :',IOU)
  print('----------------------- I AM DONE -----------------------')

In [11]:
print(tabulate([['UNet', 'VGG-16', 67, 23752273],\
                ['UNet', 'VGG-19', 70, 29061969],\
                ['UNet', 'ResNet-18', 128, 14340570],\
                ['UNet', 'ResNet-34', 200, 24456154],\
                ['UNet', 'ResNet-50', 232, 32561114],\
                ['UNet', 'ResNet-101', 419, 51605466],\
                ['UNet', 'ResNet-152', 606, 67295194],\
                ['UNet', 'DenseNet-121', 469, 12144977],\
                ['UNet', 'DenseNet-169', 637, 19519825],\
                ['UNet', 'DenseNet-201', 749, 26378577],\
                ['UNet', 'Inception-V3', 353, 29933105],\
                ['UNet', 'Inception-ResNet-V2', 822, 62061553],\
                ['UNet', 'MobileNet', 129, 8336337]],
                headers=['Model', 'Backbone', 'Number of Layers', 'Total Parameters'], tablefmt='orgtbl',stralign="center", numalign="center"))

|  Model  |      Backbone       |  Number of Layers  |  Total Parameters  |
|---------+---------------------+--------------------+--------------------|
|  UNet   |       VGG-16        |         67         |      23752273      |
|  UNet   |       VGG-19        |         70         |      29061969      |
|  UNet   |      ResNet-18      |        128         |      14340570      |
|  UNet   |      ResNet-34      |        200         |      24456154      |
|  UNet   |      ResNet-50      |        232         |      32561114      |
|  UNet   |     ResNet-101      |        419         |      51605466      |
|  UNet   |     ResNet-152      |        606         |      67295194      |
|  UNet   |    DenseNet-121     |        469         |      12144977      |
|  UNet   |    DenseNet-169     |        637         |      19519825      |
|  UNet   |    DenseNet-201     |        749         |      26378577      |
|  UNet   |    Inception-V3     |        353         |      29933105      |
|  UNet   | 

In [12]:
#------------------------- Load necessary libraries -------------------------#
from ipywidgets import widgets, Layout, Button, Box, Dropdown, Label,Output,interact, interactive_output
from IPython.display import clear_output
#------------------------- Load widgets -------------------------#
BackBoneLabel = Label(value='Select Backbone')
BackboneDrop = Dropdown(options=[('Select', ""),("VGG-16", "vgg16"),("VGG-19", "vgg19"),
                           ("ResNet-18","resnet18"),("ResNet-34","resnet34"),
                           ("ResNet-50","resnet50"),("ResNet-101","resnet101"),
                           ("ResNet-152","resnet152"),("DenseNet-121","densenet121"),
                           ("DenseNet-169","densenet169"),("DenseNet-201","densenet201"),
                           ("Inception V3","inceptionv3"),("Inception ResNet V2","inceptionresnetv2"),
                           ("MobileNet","mobilenet")],value='')
PreTrainLabel = Label(value='Use PreTrained Weights')
PreTrainedDrop = Dropdown(options=[('Yes', True),("No", False)],value=True)
SummeryLabel = Label(value='Show Model Summary')
SummeryDrop = Dropdown(options=[('Yes', True),("No", False)],value=False)
TrainLabel = Label(value='Train Model')
TrainModel = Dropdown(options=[('Yes', True),("No", False)],value=False)
TestLabel = Label(value='Test Model')
TestModel = Dropdown(options=[('Yes', True),("No", False)],value=False)
GenerateModelButton = widgets.Button(description="Generate Model",disabled=False,button_style='success',tooltip='Click to generate model')
#------------------------- Function on Selection -------------------------#
def OnSelection(BackBoneSelection,PreTrainSelection,SummerySelection,TrainSelection,TestSelection):
  PreTrainStr = None
  ShowSummaryStr = None
  TrainModelStr = None
  TestModelStr = None
  print()
  if BackBoneSelection == "":
    print("Please select a Backbone of your choice")
  else:
    PreTrainStr = "Trained" if PreTrainSelection else "UnTrained"
    ShowSummaryStr = "be" if SummerySelection else "not be"
    TrainModelStr = "be" if TrainSelection else "not be"
    TestModelStr = "be" if TestSelection else "not be"
    print("UNet Model with",PreTrainStr,BackBoneSelection,"backbone will be loaded, Model Summary will",ShowSummaryStr,"displayed, will", TrainModelStr, "trained and will",TestModelStr, "tested with new data.")
    print()
    print("If OK with above configuration, Click Generate Model (OR) Choose required configuration from GUI")
    print()
    display(GenerateModelButton)
#------------------------- Function at Button click -------------------------#
def on_button_clicked(b):
  create_model(Backbone=BackboneDrop.value,pretrained=PreTrainedDrop.value,ShowSummery=SummeryDrop.value,TrainModel=TrainModel.value,TestModel=TestModel.value)
GenerateModelButton.on_click(on_button_clicked)
#------------------------- Make widget -------------------------#
out = interactive_output(OnSelection, {"BackBoneSelection":BackboneDrop, "PreTrainSelection":PreTrainedDrop, "SummerySelection":SummeryDrop, "TrainSelection":TrainModel, "TestSelection":TestModel})
form_item_layout = Layout(display='flex',flex_flow='row',justify_content='space-between')
form_items = [
    Box([BackBoneLabel,BackboneDrop], layout=form_item_layout),
    Box([PreTrainLabel, PreTrainedDrop], layout=form_item_layout),
    Box([SummeryLabel, SummeryDrop], layout=form_item_layout),
    Box([TrainLabel, TrainModel], layout=form_item_layout),
    Box([TestLabel, TestModel], layout=form_item_layout)]
GUI = Box(form_items, layout=Layout(display='flex',flex_flow='column',border='solid 2px',align_items='stretch',width='27%'))

In [13]:
display(GUI, out)

Box(children=(Box(children=(Label(value='Select Backbone'), Dropdown(options=(('Select', ''), ('VGG-16', 'vgg1…

Output()

In [14]:
create_model(Backbone='mobilenet' ,pretrained=True,ShowSummery=False,TrainModel=True,ImageGeneratorFlow='From_Folder',TestModel=True)

-------------------Generating UNet Model----------------------

| Header               | Value     |
|----------------------+-----------|
| Model                | UNet      |
| BackBone             | mobilenet |
| Number of Layers     | 128       |
| Total params         | 8336337   |
| Trainable params     | 8312465   |
| Non-trainable params | 23872     |

-------------------Training Model----------------------

Found 21348 images belonging to 3 classes.
Found 21348 images belonging to 3 classes.
Found 5336 images belonging to 3 classes.
Found 5336 images belonging to 3 classes.

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


 51/333 [===>..........................] - ETA: 33:03 - loss: 0.1461 - accuracy: 0.9716 - mean_io_u: 0.4812 - dice_coefficient: 0.2777

Process ForkPoolWorker-20:
Process ForkPoolWorker-12:
Process ForkPoolWorker-2:
Process ForkPoolWorker-14:
Process ForkPoolWorker-17:
Process ForkPoolWorker-9:
Process ForkPoolWorker-16:
Process ForkPoolWorker-3:
Process ForkPoolWorker-22:
Process ForkPoolWorker-8:
Process ForkPoolWorker-30:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Process ForkPoolWorker-26:
Traceback (most recent call last):
Process ForkPoolWorker-24:
Process ForkPoolWorker-19:
Traceback (most recent call last):
Process ForkPoolWorker-29:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Process ForkPoolWorker-25:
Process ForkPoolWorker-1:
Process ForkPoolWorker-28:
Process ForkPoolWorker-5:
Process ForkPoolWorker-21:
Process ForkPoolWorker-23:
Process ForkPoolWorker-11:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-13:
Process 

KeyboardInterrupt: ignored

In [ ]:
create_model(Backbone='densenet121' ,pretrained=True,ShowSummery=False,TrainModel=True,ImageGeneratorFlow='From_Folder',TestModel=True)

In [ ]:
create_model(Backbone='resnet50' ,pretrained=True,ShowSummery=False,TrainModel=True,ImageGeneratorFlow='From_Folder',TestModel=True)

In [ ]:
create_model(Backbone='inceptionv3' ,pretrained=True,ShowSummery=False,TrainModel=True,ImageGeneratorFlow='From_Folder',TestModel=True)

In [ ]:
create_model(Backbone='vgg19' ,pretrained=True,ShowSummery=False,TrainModel=True,ImageGeneratorFlow='From_Folder',TestModel=True)

In [ ]:
One_Box  = '0100515c-5204-4f31-98e0-f35e4b00004a.png'	# One Box
Two_Box = '00436515-870c-4b36-a041-de91049b9ab4.png'	# Two Box
Three_Box = '03a9498c-549d-4e7d-800b-e74797f7f625.png'	# Three Box
Four_Box = '0d5bc737-03de-4bb8-98a1-45b7180c3e0f.png'	# Four Box
Not_Normal = '0004cfab-14fd-4e49-80ba-63a80b6bddd6.png'	# No Lung Opacity/Not Normal
Normal = '003d8fa0-6bf1-40ed-b54c-ac657f8495c5.png'	# Normal
test_id = Two_Box

Backbone = 'mobilenet'
preprocess_input = get_preprocessing(Backbone)
test_img_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

input_img = tf.keras.preprocessing.image.load_img(RawImg+'/'+Four_Box,target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),color_mode="rgb")
input_img = keras.preprocessing.image.img_to_array(input_img)
image = np.array([input_img])  # Convert single image to a batch.

input_mask = tf.keras.preprocessing.image.load_img(RawMask+'/'+Four_Box,target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), color_mode="grayscale")
input_mask = keras.preprocessing.image.img_to_array(input_mask,)
mask = np.array([input_mask])  # Convert single image to a batch.

filepath = '/content/drive/My Drive/great learning/capstone project/rsna-pneumonia-detection/UNet_Model_Weights/' + Backbone + "/"
WEIGHTS_FILE = filepath + "model-0.11.h5"
THRESHOLD = 0.8
EPSILON = 0.02

model = Unet(Backbone, input_shape=(224, 224, 3))
model.load_weights(WEIGHTS_FILE)

region = model.predict(image)
# region = np.where(region<threshold,0,1)
print(region.shape)
print('Max pixal value :',np.amax(region),'\nMin pixal Value :',np.amin(region))

PredictMask = region[0][:,:,0]
#PredictMask = 1.0*(PredictMask>0.5)
f, ((ax1, ax2, ax3)) = plt.subplots(1, 3, figsize = (15, 8))
ax1.imshow(image[0][:,:,0], cmap = plt.cm.bone)
ax1.set_title('Input Image')
ax1.axis('off')
ax2.imshow(mask[0][:,:,0], cmap = plt.cm.bone)
ax2.set_title('Ground Truth Mask')
ax2.axis('off')
ax3.imshow(PredictMask, cmap = plt.cm.bone)
ax3.set_title('Predicted Mask (no threshold applied)')
ax3.axis('off')
plt.show()